<a href="https://colab.research.google.com/github/zaidalyafeai/Notebooks/blob/master/TF_Swift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>![alt text](https://pbs.twimg.com/media/D1AKlRnWsAAnyP7.png)</center>

Swift for TensorFlow is a next generation platform for deep learning and differentiable programming.

In general Swift for TF tries to resolve the drawbacks of Python

*   Performance
*   Concurrency
*   Deployment
*   Custom Ops

Swift has many advantages 

*   First-class autodiff
*   Next-generation APIs
*   High-quality tooling
*   Python Integerability
*   Performance of graphs + flexibility  of Eager execution



## Constants and Variables

In [0]:
let x = 2.0 //constant
var y = 2   //variable


//define types inline 
let w:Double = 2.0
var z:Float = 3.0

In [2]:
"I have \(y) apples."

"I have 2 apples."


## Arrays and Dictionaries 

Initialize arrays and dictionaries

In [0]:
var numbersArry = [1, 2]
var numbersDict = ["one":1, "two":2]

You can also define empty arrays and dictionaries

In [4]:
var langs = [String]()

langs.append("Python")
langs.append("Csharp")
langs.append("Buisct") //not sure if that exists 
langs

▿ 3 elements
  - 0 : "Python"
  - 1 : "Csharp"
  - 2 : "Buisct"


In [5]:
var gpaStudent  = [String:Float]()

gpaStudent["Zaid"] = 3.8
gpaStudent["Zyan"] = 3.9
gpaStudent

▿ 2 elements
  ▿ 0 : 2 elements
    - key : "Zaid"
    - value : 3.8
  ▿ 1 : 2 elements
    - key : "Zyan"
    - value : 3.9


## Loops

In [6]:
let numbers = [1, 2, 3, 4]

for x in numbers {
  if x > 2{
    print("\(x) > 2")
  }
}

3 > 2
4 > 2


In [7]:
var n = 2
while n < 100 {
    n = n * 2
}

n

128


In [8]:
var total = 0
for i in 0...4 {
    total += i
}

total

10


## Functions

Here we define a function that adds two integers. By using _ we don't have to use the variable names for function calls 

In [9]:
func add(_ x: Int , _ y: Int) -> Int{
  return x + y
}
print(add(2, 3))

5


In [0]:
func mean_variance(scores: [Double]) -> (mean: Double, variance: Double){
  
  var mean: Double = 0
  var variance: Double = 0
  let count: Double = Double(scores.count)
  
  for x in scores{
    mean += (x/count)
  }
  
  for x in scores{
    variance += (x - mean)*(x - mean)/count
  }
  
  return (mean, variance)
}

In [11]:
mean_variance(scores: [2.0 , 3.0 , 4.0])

▿ 2 elements
  - mean : 3.0
  - variance : 0.6666666666666666


## Objects and Classes


In [0]:
class Student {
  var name: String = ""
  var age : Int = 0 
  
  init(name: String, age: Int){
    self.name = name
    self.age = age 
  }
}

In [13]:
let person = Student(name:"Zaid", age: 3)
print(person.name)

Zaid


## Generics

Generics are used to construct generic types. Use `<>` to represent a generic type 

In [14]:
//convert a list of generic type to a dictionary of generic type
func makeDict<T>(_ array: [T]) -> [T:T] {
  var out = [T:T]()
  
  
  for a in array{
    out[a] = a
  }
  return out
}

makeDict([2, 3, 4])

▿ 3 elements
  ▿ 0 : 2 elements
    - key : 3
    - value : 3
  ▿ 1 : 2 elements
    - key : 4
    - value : 4
  ▿ 2 : 2 elements
    - key : 2
    - value : 2


## Python Integeration

It is easy to call python modules in Swift

In [0]:
import Python
let np = Python.import("numpy")

In [16]:
np.exp(0)

1.0


## Differentiation

We can easily define a differentiable function by using `@differentiable`

In [0]:
@differentiable
func frac(_ x:Double) -> Double{
  return 1/x
}

In [18]:
gradient(at:0.5) {x in frac(x)}

-4.0


# TenosrFlow

In [0]:
import TensorFlow

## Neural Network (NN)

We create a simple XOR classifier example using NN

### Tensors

In [0]:
var X: Tensor<Float> = [[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 1.0]]
var y: Tensor<Int32> = [0, 1, 1, 0]

### Model

Create a nerual network with one hidden layer

In [0]:
let hiddenSize: Int = 10

struct NN: Layer {
    var layer1 = Dense<Float>(inputSize: 2, outputSize: hiddenSize, activation: relu)
    var layer2 = Dense<Float>(inputSize: hiddenSize, outputSize: 2)
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: layer1, layer2)
    }
}

In [0]:
let optimizer = SGD<NN, Float>(learningRate: 0.01)

### Training

In [23]:
var model = NN()

let trainingContext = Context(learningPhase: .training)

for i in 0...1000 {
  
  //calculate the loss and gradient
  let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
      let logits = model.applied(to: X, in: trainingContext)
      return softmaxCrossEntropy(logits: logits, labels: y)
  }

  //make an optimizer step 
  optimizer.update(&model.allDifferentiableVariables, along: grads)

  if i % 100 == 0 {
    print("epoch \(i) train_loss: \(loss)")
  }
  
}

epoch 0 train_loss: 0.69633675
epoch 100 train_loss: 0.626324
epoch 200 train_loss: 0.595307
epoch 300 train_loss: 0.56721175
epoch 400 train_loss: 0.5392669
epoch 500 train_loss: 0.5136224
epoch 600 train_loss: 0.49167603
epoch 700 train_loss: 0.47081134
epoch 800 train_loss: 0.45074087
epoch 900 train_loss: 0.43119302
epoch 1000 train_loss: 0.41217735


### Inference

In [24]:
let inferenceContext = Context(learningPhase: .inference)

// Apply the model to a batch of features.
let pred = model.applied(to: X, in: inferenceContext)

print(pred.argmax(squeezingAxis: 1))

[0, 1, 1, 0]


## Convolutional Neural Network (CNN)

We create a simple classifier using a CNN 

### Tensors

In [0]:
let X = Tensor<Float>(randomNormal: [10, 16, 16, 3])
let y = Tensor<Int32>(rangeFrom: 0, to: 10, stride: 1)

### Model

In [0]:
struct CNN: Layer {
    var conv = Conv2D<Float>(filterShape: (5, 5, 3, 16)) //(w,h,c,out)
    var maxPool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var flatten = Flatten<Float>()
    var dense = Dense<Float>(inputSize: flatten., outputSize: 10)

    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: conv, maxPool, flatten, dense)
    }
}

In [0]:
let optimizer = SGD<CNN, Float>(learningRate: 0.01)

### Training

In [0]:
var model = CNN()

let trainingContext = Context(learningPhase: .training)

for i in 0...100 {
  
  //calculate the loss and gradient
  let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
      let logits = model.applied(to: X, in: trainingContext)
      return softmaxCrossEntropy(logits: logits, labels: y)
  }

  //make an optimizer step 
  optimizer.update(&model.allDifferentiableVariables, along: grads)

  if i % 10 == 0 {
    print("epoch \(i) train_loss: \(loss)")
  }
  
}

epoch 0 train_loss: 2.862069
epoch 10 train_loss: 0.9227737
epoch 20 train_loss: 0.3801387
epoch 30 train_loss: 0.21386012
epoch 40 train_loss: 0.14373973
epoch 50 train_loss: 0.10649824
epoch 60 train_loss: 0.08388945
epoch 70 train_loss: 0.068893135
epoch 80 train_loss: 0.05833138
epoch 90 train_loss: 0.050462723
epoch 100 train_loss: 0.044379223


# References



*   https://github.com/tensorflow/swift/blob/master/docs/WhySwiftForTensorFlow.md
*   https://www.tensorflow.org/swift
*   https://www.tensorflow.org/swift/tutorials/model_training_walkthrough

